### import libraries

In [1]:
import pandas as pd
import numpy as np

### load dataset

In [2]:
df = pd.read_csv("data/covid.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [3]:
#no of missing values
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [4]:
df.gender.value_counts()

Female    59
Male      41
Name: gender, dtype: int64

In [5]:
df.cough.value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [6]:
df.city.value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

### splitting data into train and test set

In [7]:
X = df.drop(columns=['has_covid'])
y = df['has_covid']

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

((80, 5), (20, 5))

### ye hai aam jindagi

In [9]:
# adding simple imputer to fever col
from sklearn.impute import SimpleImputer

si = SimpleImputer()

# train data
X_train_fever = si.fit_transform(X_train[['fever']])

# test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape, X_test_fever.shape

((80, 1), (20, 1))

In [10]:
# OneHotEncoding -> gender,city
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first',sparse=False)

#train data
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

#test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape, X_test_gender_city.shape

((80, 4), (20, 4))

In [11]:
# Ordinal encoding -> cough
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder(categories=[['Mild','Strong']])

#train data
X_train_cough = oe.fit_transform(X_train[['cough']])

#test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape, X_test_cough.shape

((80, 1), (20, 1))

In [12]:
# Extracting Age
X_train_age = X_train.loc[:,["age"]].values

# also the test data
X_test_age = X_test.loc[:,["age"]].values

X_train_age.shape, X_test_age.shape

((80, 1), (20, 1))

In [13]:
#transformed data

#train data
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)

# test data
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

X_train_transformed.shape, X_test_transformed.shape

((80, 7), (20, 7))

### ye hai ml jindagi

In [14]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers = [
                                        ('tnf1',SimpleImputer(),['fever']),
                                        ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
                                        ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
                                    ], remainder='passthrough')

transformer.fit_transform(X_train).shape

(80, 7)

In [15]:
transformer.fit_transform(X_test).shape

(20, 7)

In [16]:
transformer.fit_transform(X_train)

array([[102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [100.82857143,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  38.        ],
       [104.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  54.        ],
       [ 99.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  25.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  55.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  10.        ],
       [100.82857143,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          0.    